In [14]:
import os
import sys
import random
import warnings
import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 152
IMG_HEIGHT = 60
IMG_CHANNELS = 3
TRAIN_PATH =r'C:\Users\jmlee\Desktop\4학년 1학기\졸프\U-Net\input_train'
TEST_PATH =r'C:\Users\jmlee\Desktop\4학년 1학기\졸프\U-Net\input_test'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [15]:
#한글이름 사진 로드
def hangulFilePathImageRead (filePath ) :

    stream = open( filePath.encode("utf-8") , "rb")
    bytes = bytearray(stream.read())
    numpyArray = np.asarray(bytes, dtype=np.uint8)

    return cv2.imdecode(numpyArray , cv2.IMREAD_COLOR)  #3채널로 가져오기

if __name__ == '__main__':
    for f in os.listdir(TRAIN_PATH):  
        if f.split('.')[1] =='jpg':
            filePath = (r"C:/Users/jmlee/Desktop/4학년 1학기/졸프/U-Net/input_train"+'/'+f)  
            train_ids = hangulFilePathImageRead(filePath)

In [18]:
if __name__ == '__main__':
    for f in os.listdir(TEST_PATH):  
        if f.split('.')[1] =='jpg':
            filePath = (r"C:/Users/jmlee/Desktop/4학년 1학기/졸프/U-Net/input_test"+'/'+f)  
            test_ids = hangulFilePathImageRead(filePath)

In [21]:
type(train_ids)

numpy.ndarray

## 데이터 전처리

In [25]:
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)): #tqdm:진행바 표시
    path = TRAIN_PATH + id_
    print(path)
    img = imread(path + '/images/' + id_ + '.jpg')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.jpg')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

Getting and resizing train images and masks ... 


  0%|                                                                                           | 0/60 [00:00<?, ?it/s]

[[  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  1   1   1]
 [  2   2   2]
 [  2   2   2]
 [  3   3   3]
 [  3   3   3]
 [  7   7   7]
 [  9   9   9]
 [ 12  12  12]
 [ 13  13  13]
 [ 12  12  12]
 [ 12  12  12]
 [ 13  13  13]
 [ 15  15  15]
 [ 20  20  20]
 [ 21  21  21]
 [ 22  22  22]
 [ 24  24  24]
 [ 24  24  24]
 [ 25  25  25]
 [ 25  25  25]
 [ 25  25  25]
 [ 26  26  26]
 [ 29  29  29]
 [ 33  33  33]
 [ 38  38  38]
 [ 42  42  42]
 [ 45  45  45]
 [ 46  46  46]
 [ 47  47  47]
 [ 52  52  52]
 [ 62  62  62]
 [ 52  52  52]
 [ 39  39  39]
 [ 49  49  49]
 [ 64  64  64]
 [ 63  63  63]
 [ 59  59  59]
 [ 66  66  66]
 [ 60  60  60]
 [ 54  54  54]
 [ 54  54  54]
 [ 59  59  59]
 [ 67  67  67]
 [ 73  73  73]
 [ 76  76  76]
 [ 73  73  73]
 [ 70  70  70]
 [ 69  69  69]
 [ 70  70  70]
 [ 64  64  64]
 [ 55  55  55]
 [ 55  55  55]
 [ 61  61  61]
 [ 57  57  57]
 [ 58  58  58]
 [ 60  60  60]
 [ 61  61  61]
 [ 63  63  63]
 [ 64  64  64]
 [ 64  64  64]
 [ 64  64  64]
 [ 64  64  64]
 [ 65  65  65]
 [ 69  69 

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U51'), dtype('<U51')) -> dtype('<U51')